# Lab 02b: Training a CNN on Synthetic Handwriting Data

### What You Will Learn

- Fundamental principles for building neural networks with convolutional components
- How to use Lightning's training framework via a CLI

## Setup

In [1]:
if "bootstrap" not in locals() or bootstrap.run:
    # path management for Python
    pythonpath, = !echo $PYTHONPATH
    if "." not in pythonpath.split(":"):
        pythonpath = ".:" + pythonpath
        %env PYTHONPATH={pythonpath}
        !echo $PYTHONPATH

    # get both Colab and local notebooks into the same state
    !wget --quiet https://fsdl.me/gist-bootstrap -O bootstrap.py
    import bootstrap

    # change into the lab directory
    # bootstrap.change_to_lab_dir(lab_idx=lab_idx)

    # allow "hot-reloading" of modules
    %load_ext autoreload
    %autoreload 2
    # needed for inline plots in some contexts
    %matplotlib inline

    bootstrap.run = False  # change to True re-run setup
    
!pwd
%ls

/home/amazingguni/git/fsdl-text-recognizer-2022-labs
Makefile   data/            lightning_logs/  requirements/     training/
README.md  environment.yml  notebooks/       text_recognizer/


## Why convolutions

In [2]:
import random

import IPython.display as display

randsize = 10 ** (random.random() * 2 + 1)

Url = "https://fsdl-public-assets.s3.us-west-2.amazonaws.com/emnist/U.png"

# run multiple times to display the same image at different sizes
#  the content of the image remains unambiguous
display.Image(url=Url, width=randsize, height=randsize)

## Convolutions are the local, translation-equivariant linear transforms.

In [3]:
import torch


generic_linear_transform = torch.randn(8, 1)
print("generic:", generic_linear_transform, sep="\n")

local_linear_transform = torch.tensor([
    [0, 0, 0] + [random.random(), random.random(), random.random()] + [0, 0]]).T
print("local:", local_linear_transform, sep="\n")

generic:
tensor([[ 0.1797],
        [ 0.5482],
        [ 1.1588],
        [-0.4887],
        [-0.0285],
        [-0.4871],
        [ 0.7276],
        [ 0.4638]])
local:
tensor([[0.0000],
        [0.0000],
        [0.0000],
        [0.3999],
        [0.4685],
        [0.0765],
        [0.0000],
        [0.0000]])


In [4]:
generic_linear_transform = torch.arange(8)[:, None]
print("generic:", generic_linear_transform, sep="\n")

equivariant_linear_transform = torch.stack([torch.roll(generic_linear_transform[:, 0], ii) for ii in range(8)], dim=1)
print("translation invariant:", equivariant_linear_transform, sep="\n")

generic:
tensor([[0],
        [1],
        [2],
        [3],
        [4],
        [5],
        [6],
        [7]])
translation invariant:
tensor([[0, 7, 6, 5, 4, 3, 2, 1],
        [1, 0, 7, 6, 5, 4, 3, 2],
        [2, 1, 0, 7, 6, 5, 4, 3],
        [3, 2, 1, 0, 7, 6, 5, 4],
        [4, 3, 2, 1, 0, 7, 6, 5],
        [5, 4, 3, 2, 1, 0, 7, 6],
        [6, 5, 4, 3, 2, 1, 0, 7],
        [7, 6, 5, 4, 3, 2, 1, 0]])


In [5]:
# the equivalent of torch.nn.Linear, but for a 1-dimensional convolution
conv_layer = torch.nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3)

conv_layer.weight  # aka kernel

Parameter containing:
tensor([[[ 0.2425,  0.5438, -0.2380]]], requires_grad=True)

In [6]:
conv_kernel_as_vector = torch.hstack([conv_layer.weight[0][0], torch.zeros(5)])
conv_layer_as_matrix = torch.stack([torch.roll(conv_kernel_as_vector, ii) for ii in range(8)], dim=0)
print("convolution matrix:", conv_layer_as_matrix, sep="\n")

convolution matrix:
tensor([[ 0.2425,  0.5438, -0.2380,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.2425,  0.5438, -0.2380,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.2425,  0.5438, -0.2380,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.2425,  0.5438, -0.2380,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.2425,  0.5438, -0.2380,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.2425,  0.5438, -0.2380],
        [-0.2380,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.2425,  0.5438],
        [ 0.5438, -0.2380,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.2425]],
       grad_fn=<StackBackward0>)


In [7]:
display.Image(
    url="https://upload.wikimedia.org/wikipedia/commons/5/56/RGB_channels_separation.png?20110219015028")

In [8]:
# a sample image
display.Image(url="https://distill.pub/2018/building-blocks/examples/input_images/dog_cat.jpeg")

In [9]:
# from https://distill.pub/2018/building-blocks/
display.Image(url="https://fsdl-public-assets.s3.us-west-2.amazonaws.com/distill-feature-attrib.png", width=1024)

In [10]:
layers = ["conv2d0", "conv2d1", "conv2d2", "mixed3a", "mixed3b"]
layer = layers[1]
idx = 52

weight_explorer = display.IFrame(
    src=f"https://storage.googleapis.com/distill-circuits/inceptionv1-weight-explorer/{layer}_{idx}.html", width=1024, height=720)
weight_explorer.iframe = 'style="background: #FFF";\n><'.join(weight_explorer.iframe.split("><"))  # inject background color
weight_explorer

## Applying convolutions to handwritten characters: `CNN`s on `EMNIST`

In [11]:
import text_recognizer.models


text_recognizer.models.CNN??

Init signature:
text_recognizer.models.CNN(
    data_config: Dict[str, Any],
    args: argparse.Namespace = None,
) -> None
Source:        
class CNN(nn.Module):
    """Simple CNN for recognizing characters in a square image."""

    def __init__(self, data_config: Dict[str, Any], args: argparse.Namespace = None) -> None:
        super().__init__()
        self.args = vars(args) if args is not None else {}
        self.data_config = data_config

        input_channels, input_height, input_width = self.data_config["input_dims"]
        assert (
            input_height == input_width
        ), f"input height and width should be equal, but was {input_height}, {input_width}"
        self.input_height, self.input_width = input_height, input_width

        num_classes = len(self.data_config["mapping"])

        conv_dim = self.args.get("conv_dim", CONV_DIM)
        fc_dim = self.args.get("fc_dim", FC_DIM)
        fc_dropout = self.args.get("fc_dropout", FC_DROPOUT)

        self.conv1 = Co

## The `EMNIST` Handwritten Character Dataset

In [12]:
import text_recognizer.data

emnist = text_recognizer.data.EMNIST()
print(emnist.__doc__)


EMNIST dataset of handwritten characters and digits.

    "The EMNIST dataset is a set of handwritten character digits derived from the NIST Special Database 19
    and converted to a 28x28 pixel image format and dataset structure that directly matches the MNIST dataset."
    From https://www.nist.gov/itl/iad/image-group/emnist-dataset

    The data split we will use is
    EMNIST ByClass: 814,255 characters. 62 unbalanced classes.
    


In [13]:
emnist.prepare_data()  # download, save to disk
emnist.setup()  # create torch.utils.data.Datasets, do train/val split

In [14]:
!echo {emnist.data_dirname()}
!ls {emnist.data_dirname()}
!ls {emnist.data_dirname() / "raw" / "emnist"}

/home/amazingguni/git/fsdl-text-recognizer-2022-labs/data
downloaded  processed  raw
metadata.toml  readme.md


In [15]:
emnist

EMNIST Dataset
Num classes: 83
Mapping: ['<B>', '<S>', '<E>', '<P>', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '!', '"', '#', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '?']
Dims: (1, 28, 28)
Train/val/test sizes: 260212, 65054, 53988
Batch x stats: (torch.Size([128, 1, 28, 28]), torch.float32, tensor(0.), tensor(0.1672), tensor(0.3262), tensor(1.))
Batch y stats: (torch.Size([128]), torch.int64, tensor(4), tensor(65))

In [16]:
xs, ys = next(iter(emnist.train_dataloader()))

In [17]:
import wandb

idx = random.randint(0, len(xs) - 1)

print(emnist.mapping[ys[idx]])
wandb.Image(xs[idx]).image

l


## Putting convolutions in a `torch.nn.Module`

In [18]:
data_config = emnist.config()

cnn = text_recognizer.models.CNN(data_config)
cnn

CNN(
  (conv1): ConvBlock(
    (conv): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
  )
  (conv2): ConvBlock(
    (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
  )
  (dropout): Dropout(p=0.25, inplace=False)
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=12544, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=83, bias=True)
)

In [19]:
idx = random.randint(0, len(xs) - 1)
outs = cnn(xs[idx: idx + 1])
print('output:', emnist.mapping[torch.argmax(outs)])
wandb.Image(xs[idx]).image

output: m


In [20]:
cnn.forward??

Signature: cnn.forward(x: torch.Tensor) -> torch.Tensor
Source:   
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Applies the CNN to x.

        Parameters
        ----------
        x
            (B, Ch, H, W) tensor, where H and W must equal input height and width from data_config.

        Returns
        -------
        torch.Tensor
            (B, Cl) tensor
        """
        _B, _Ch, H, W = x.shape
        assert H == self.input_height and W == self.input_width, f"bad inputs to CNN with shape {x.shape}"
        x = self.conv1(x)  # _B, CONV_DIM, H, W
        x = self.conv2(x)  # _B, CONV_DIM, H, W
        x = self.max_pool(x)  # _B, CONV_DIM, H // 2, W // 2
        x = self.dropout(x)
        x = torch.flatten(x, 1)  # _B, CONV_DIM * H // 2 * W // 2
        x = self.fc1(x)  # _B, FC_DIM
        x = F.relu(x)
        x = self.fc2(x)  # _B, Cl
        return x
File:      ~/git/fsdl-text-recognizer-2022-labs/text_recognizer/models/cnn.py
Type:      method


In [21]:
torch.flatten(torch.Tensor([[1, 2], [3, 4]]))

tensor([1., 2., 3., 4.])

In [22]:
[p.numel() for p in cnn.parameters()]  # conv weight + bias, conv weight + bias, fc weight + bias, fc weight + bias

[576, 64, 36864, 64, 1605632, 128, 10624, 83]

In [23]:
biggest_layer = [p for p in cnn.parameters() if p.numel() == max(p.numel() for p in cnn.parameters())][0]
biggest_layer.shape, cnn.fc_input_dim

(torch.Size([128, 12544]), 12544)

In [24]:
# for the Linear layers, number of multiplications per input == nparams
cnn.fc1.weight.numel()

1605632

In [25]:
# for the Conv2D layers, it's more complicated

def approx_conv_multiplications(kernel_shape, input_size=(64, 28, 28)):  # this is a rough and dirty approximation
    num_kernel_elements = 1
    for dimension in kernel_shape[-3:]:
        num_kernel_elements *= dimension
    num_input_channels, num_kernels = input_size[0], kernel_shape[0]
    num_spatial_applications = ((input_size[1] - kernel_shape[-2] + 1) * (input_size[2] - kernel_shape[-1] + 1))
    mutliplications_per_kernel = num_spatial_applications * num_kernel_elements * num_input_channels
    return mutliplications_per_kernel * num_kernels

In [26]:
approx_conv_multiplications(cnn.conv2.conv.weight.shape)

1594884096

In [27]:
# ratio of multiplications in the convolution to multiplications in the fully-connected layer is huge!
approx_conv_multiplications(cnn.conv2.conv.weight.shape) // cnn.fc1.weight.numel()

993

## Training a `CNN` on `EMNIST` with the Lightning `Trainer` and `run_experiment`

In [28]:
!python training/run_experiment.py --help --model_class CNN --data_class EMNIST  | tail -n 25

                        .. deprecated:: v1.5 Trainer argument
                        ``terminate_on_nan`` was deprecated in v1.5 and will
                        be removed in 1.7. Please use ``detect_anomaly``
                        instead.

Data Args:
  --batch_size BATCH_SIZE
                        Number of examples to operate on per forward step.
                        Default is 128.
  --num_workers NUM_WORKERS
                        Number of additional processes to load data. Default
                        is 16.

Model Args:
  --conv_dim CONV_DIM
  --fc_dim FC_DIM
  --fc_dropout FC_DROPOUT

LitModel Args:
  --optimizer OPTIMIZER
                        optimizer class from torch.optim
  --lr LR
  --one_cycle_max_lr ONE_CYCLE_MAX_LR
  --one_cycle_total_steps ONE_CYCLE_TOTAL_STEPS
  --loss LOSS           loss function from torch.nn.functional


In [29]:
import training.run_experiment


print(training.run_experiment.main.__doc__)


    Run an experiment.

    Sample command:
    ```
    python training/run_experiment.py --max_epochs=3 --gpus='0,' --num_workers=20 --model_class=MLP --data_class=MNIST
    ```

    For basic help documentation, run the command
    ```
    python training/run_experiment.py --help
    ```

    The available command line args differ depending on some of the arguments, including --model_class and --data_class.

    To see which command line args are available and read their documentation, provide values for those arguments
    before invoking --help, like so:
    ```
    python training/run_experiment.py --model_class=MLP --data_class=MNIST --help
    


In [30]:
gpus = int(torch.cuda.is_available())  # use GPUs if they're available

%run training/run_experiment.py --model_class CNN --data_class EMNIST --gpus {gpus}

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type      | Params
---------------------------------------------
0 | model          | CNN       | 1.7 M 
1 | model.conv1    | ConvBlock | 640   
2 | model.conv2    | ConvBlock | 36.9 K
3 | model.dropout  | Dropout   | 0     
4 | model.max_pool | MaxPool2d | 0     
5 | model.fc1      | Linear    | 1.6 M 
6 | model.fc2      | Linear    | 10.7 K
7 | train_acc      | Accuracy  | 0     
8 | val_acc        | Accuracy  | 0     
9 | test_acc       | Accuracy  | 0     
---------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.616     Total estima

Model State Dict Disk Size: 6.62 MB


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Best model saved at: /home/amazingguni/git/fsdl-text-recognizer-2022-labs/training/logs/lightning_logs/version_3/epoch=0000-validation.loss=0.574.ckpt
Restoring states from the checkpoint path at /home/amazingguni/git/fsdl-text-recognizer-2022-labs/training/logs/lightning_logs/version_3/epoch=0000-validation.loss=0.574.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /home/amazingguni/git/fsdl-text-recognizer-2022-labs/training/logs/lightning_logs/version_3/epoch=0000-validation.loss=0.574.ckpt


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/acc            0.7846002578735352
        test/loss           0.5775994062423706
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [31]:
# we use a sequence of bash commands to get the latest checkpoint's filename
#  by hand, you can just copy and paste it

list_all_log_files = "find training/logs/lightning_logs"  # find avoids issues with \n in filenames
filter_to_ckpts = "grep \.ckpt$"  # regex match on end of line
sort_version_descending = "sort -Vr"  # uses "version" sorting (-V) and reverses (-r)
take_first = "head -n 1"  # the first n elements, n=1

latest_ckpt, = ! {list_all_log_files} | {filter_to_ckpts} | {sort_version_descending} | {take_first}
latest_ckpt

'training/logs/lightning_logs/version_3/epoch=0000-validation.loss=0.574.ckpt'

In [32]:
import training.util
from argparse import Namespace


# if you change around model/data args in the command above, add them here
#  tip: define the arguments as variables, like we've done for gpus
#       and then add those variables to this dict so you don't need to
#       remember to update/copy+paste

args = Namespace(**{
    "model_class": "CNN",
    "data_class": "EMNIST"})


_, cnn = training.util.setup_data_and_model_from_args(args)

reloaded_model = text_recognizer.lit_models.BaseLitModel.load_from_checkpoint(
   latest_ckpt, args=args, model=cnn)

In [33]:
idx = random.randint(0, len(xs) - 1)
outs = reloaded_model(xs[idx:idx+1])

print("output:", emnist.mapping[torch.argmax(outs)])
wandb.Image(xs[idx]).image

output: l


In [34]:
latest_ckpt, = ! {list_all_log_files} | {filter_to_ckpts} | {sort_version_descending} | {take_first}


# and we can change the training hyperparameters, like batch size
%run training/run_experiment.py --model_class CNN --data_class EMNIST --gpus {gpus} \
  --batch_size 64 --load_checkpoint {latest_ckpt}

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type      | Params
---------------------------------------------
0 | model          | CNN       | 1.7 M 
1 | model.conv1    | ConvBlock | 640   
2 | model.conv2    | ConvBlock | 36.9 K
3 | model.dropout  | Dropout   | 0     
4 | model.max_pool | MaxPool2d | 0     
5 | model.fc1      | Linear    | 1.6 M 
6 | model.fc2      | Linear    | 10.7 K
7 | train_acc      | Accuracy  | 0     
8 | val_acc        | Accuracy  | 0     
9 | test_acc       | Accuracy  | 0     
---------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.616     Total estima

Model State Dict Disk Size: 6.62 MB


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Best model saved at: /home/amazingguni/git/fsdl-text-recognizer-2022-labs/training/logs/lightning_logs/version_4/epoch=0000-validation.loss=0.532.ckpt
Restoring states from the checkpoint path at /home/amazingguni/git/fsdl-text-recognizer-2022-labs/training/logs/lightning_logs/version_4/epoch=0000-validation.loss=0.532.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /home/amazingguni/git/fsdl-text-recognizer-2022-labs/training/logs/lightning_logs/version_4/epoch=0000-validation.loss=0.532.ckpt


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/acc            0.7939727306365967
        test/loss           0.5348023176193237
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


## Creating lines of text from handwritten characters: `EMNISTLines`

In [35]:
from text_recognizer.data.sentence_generator import SentenceGenerator

sentence_generator = SentenceGenerator()

SentenceGenerator.__doc__

'Generate text sentences using the Brown corpus.'

In [36]:
print(*[sentence_generator.generate(max_length=16) for _ in range(4)], sep="\n")

phase
Within
either
the


In [41]:
emnist_lines = text_recognizer.data.EMNISTLines()  # configure
emnist_lines.__doc__

'EMNIST Lines dataset: synthetic handwriting lines dataset made from EMNIST characters.'

In [42]:
emnist_lines.prepare_data()  # download, save to disk
emnist_lines.setup()  # create torch.utils.data.Datasets, do train/val split
emnist_lines

EMNISTLinesDataset generating data for train...
EMNISTLinesDataset generating data for val...
EMNISTLinesDataset generating data for test...
EMNISTLinesDataset loading data from HDF5...


EMNIST Lines Dataset
Min overlap: 0
Max overlap: 0.33
Num classes: 83
Dims: (1, 28, 896)
Output dims: (32, 1)
Train/val/test sizes: 10000, 2000, 2000
Batch x stats: (torch.Size([128, 1, 28, 896]), torch.float32, 0.0, 0.07364349067211151, 0.23208004236221313, 1.0)
Batch y stats: (torch.Size([128, 32]), torch.int64, 3, 66)

In [43]:
line_xs, line_ys = next(iter(emnist_lines.val_dataloader()))
line_xs.shape, line_ys.shape

(torch.Size([128, 1, 28, 896]), torch.Size([128, 32]))

In [48]:
def read_line_labels(labels):
    return [emnist_lines.mapping[label] for label in labels]

idx = random.randint(0, len(line_xs) - 1)

print("-".join(read_line_labels(line_ys[idx])))
wandb.Image(line_xs[idx]).image

f-e-e-t- -H-e- -h-o-v-e-r-e-d- -o-v-e-r- -h-e-r-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>


## Applying CNNs to handwritten text: `LineCNNSimple`

In [49]:
line_cnn = text_recognizer.models.LineCNNSimple(emnist_lines.config())
line_cnn

LineCNNSimple(
  (cnn): CNN(
    (conv1): ConvBlock(
      (conv): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu): ReLU()
    )
    (conv2): ConvBlock(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu): ReLU()
    )
    (dropout): Dropout(p=0.25, inplace=False)
    (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (fc1): Linear(in_features=12544, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=83, bias=True)
  )
)

In [50]:
line_cnn.forward??

Signature: line_cnn.forward(x: torch.Tensor) -> torch.Tensor
Source:   
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Apply the LineCNN to an input image and return logits.

        Parameters
        ----------
        x
            (B, C, H, W) input image with H equal to IMAGE_SIZE

        Returns
        -------
        torch.Tensor
            (B, C, S) logits, where S is the length of the sequence and C is the number of classes
            S can be computed from W and CHAR_WIDTH
            C is self.num_classes
        """
        B, _C, H, W = x.shape
        assert H == IMAGE_SIZE  # Make sure we can use our CNN class

        # Compute number of windows
        S = math.floor((W - self.WW) / self.WS + 1)

        # NOTE: type_as properly sets device
        activations = torch.zeros((B, self.num_classes, S)).type_as(x)
        for s in range(S):
            start_w = self.WS * s
            end_w = start_w + self.WW
            window = x[:, :, :, start_

In [51]:
idx = random.randint(0, len(line_xs) - 1)

outs, = line_cnn(line_xs[idx:idx+1])
preds = torch.argmax(outs, 0)

print("-".join(read_line_labels(preds)))
wandb.Image(line_xs[idx]).image

H-K-K-K-k-k-H-H-K-K-H-N-H-H-N-k-s-H-q-N-L-N-q-q-H-N-N-N-N-q-H-H


In [52]:
%run training/run_experiment.py --model_class LineCNNSimple --data_class EMNISTLines \
  --batch_size 32 --gpus {gpus} --max_epochs 2

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


EMNISTLinesDataset loading data from HDF5...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type          | Params
--------------------------------------------
0 | model     | LineCNNSimple | 1.7 M 
1 | model.cnn | CNN           | 1.7 M 
2 | train_acc | Accuracy      | 0     
3 | val_acc   | Accuracy      | 0     
4 | test_acc  | Accuracy      | 0     
--------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.616     Total estimated model params size (MB)


Model State Dict Disk Size: 6.62 MB


Sanity Checking: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Best model saved at: /home/amazingguni/git/fsdl-text-recognizer-2022-labs/training/logs/lightning_logs/version_5/epoch=0001-validation.loss=1.372.ckpt
Restoring states from the checkpoint path at /home/amazingguni/git/fsdl-text-recognizer-2022-labs/training/logs/lightning_logs/version_5/epoch=0001-validation.loss=1.372.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at /home/amazingguni/git/fsdl-text-recognizer-2022-labs/training/logs/lightning_logs/version_5/epoch=0001-validation.loss=1.372.ckpt


EMNISTLinesDataset loading data from HDF5...


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/acc            0.6771718859672546
        test/loss           1.4252547025680542
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [53]:
# if you change around model/data args in the command above, add them here
#  tip: define the arguments as variables, like we've done for gpus
#       and then add those variables to this dict so you don't need to
#       remember to update/copy+paste

args = Namespace(**{
    "model_class": "LineCNNSimple",
    "data_class": "EMNISTLines"})


_, line_cnn = training.util.setup_data_and_model_from_args(args)

latest_ckpt, = ! {list_all_log_files} | {filter_to_ckpts} | {sort_version_descending} | {take_first}
print(latest_ckpt)

reloaded_lines_model = text_recognizer.lit_models.BaseLitModel.load_from_checkpoint(
   latest_ckpt, args=args, model=line_cnn)

training/logs/lightning_logs/version_5/epoch=0001-validation.loss=1.372.ckpt


In [56]:
idx = random.randint(0, len(line_xs) - 1)

outs, = reloaded_lines_model(line_xs[idx:idx+1])
preds = torch.argmax(outs, 0)

print("-".join(read_line_labels(preds)))
wandb.Image(line_xs[idx]).image

i-s- -m-o-r-e- -e-l-e- -l-i-i-e-o-f-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>-<P>


In [57]:
padding_token = emnist_lines.emnist.inverse_mapping["<P>"]
torch.sum(line_ys == padding_token) / line_ys.numel()

tensor(0.5359)